In [1]:
from huggingface_hub import hf_hub_download
import os
import tarfile
from pathlib import Path

from utils_agibot import *

In [40]:
REPO_ID = "agibot-world/AgiBotWorld-Beta"
TASK_ID = "429"  # Change to your desired task
OBSERVATION_FILE = f"observations/{TASK_ID}/747793-785810.tar"  # Find yours from the link above
LOCAL_DIR = f"/shared_work/physical_intelligence/agibot_world_sample/{TASK_ID}"
# Create directory if it doesn't exist
os.makedirs(LOCAL_DIR, exist_ok=True)

In [30]:
# # Download observation tar (only if not exists)
# obs_path = Path(LOCAL_DIR) / OBSERVATION_FILE
# if not obs_path.exists():
#     print(f"Downloading observation tar...")
#     obs_path = hf_hub_download(
#         repo_id=REPO_ID,
#         filename=OBSERVATION_FILE,
#         repo_type="dataset",
#         local_dir=LOCAL_DIR
#     )
#     print(f"Downloaded observation: {obs_path}")
# else:
#     print(f"Observation already exists: {obs_path}")

# # Download task info JSON (only if not exists)
# task_info_path = Path(LOCAL_DIR) / f"task_info/task_{TASK_ID}.json"
# if not task_info_path.exists():
#     print(f"Downloading task info...")
#     task_info_path = hf_hub_download(
#         repo_id=REPO_ID,
#         filename=f"task_info/task_{TASK_ID}.json",
#         repo_type="dataset",
#         local_dir=LOCAL_DIR
#     )
#     print(f"Downloaded task info: {task_info_path}")
# else:
#     print(f"Task info already exists: {task_info_path}")

In [31]:
# # Extract the tar file
# target_dir = Path(LOCAL_DIR) / "observations" / TASK_ID
# tar_file = list(target_dir.glob("*.tar"))[0]

# print(f"Extracting {tar_file}...")
# with tarfile.open(tar_file, 'r') as tar:
#     tar.extractall(path=target_dir)
# print(f"Extraction complete to {target_dir}")

In [41]:
episode_id = 684445
json_path = f"task_info/task_{TASK_ID}.json"
task_info_path = os.path.join(LOCAL_DIR, json_path)
video_head = f'observations/{TASK_ID}/{episode_id}/videos/head_color.mp4'
video_hand_right = f'observations/{TASK_ID}/{episode_id}/videos/hand_right_color.mp4'
video_hand_left = f'observations/{TASK_ID}/{episode_id}/videos/hand_left_color.mp4'

video_path_head = os.path.join(LOCAL_DIR,video_head)
video_path_hand_right = os.path.join(LOCAL_DIR,video_hand_right)
video_path_hand_left = os.path.join(LOCAL_DIR,video_hand_left)

In [42]:
episode_info = load_action_config(task_info_path, episode_id)

print(f"Episode ID: {episode_info['episode_id']}")
print(f"Task Name: {episode_info['task_name']}")
print(f"\nAction Config:")

action_config = episode_info.get('label_info', {}).get('action_config', [])
for i, action in enumerate(action_config):
    print(f"  {i}: frames {action['start_frame']}-{action['end_frame']}: {action.get('action_text', 'N/A')}")

Episode ID: 684445
Task Name: Install memory module

Action Config:
  0: frames 0-206: Press the latch of the memory card slot with your right arm.
  1: frames 206-711: Insert the memory stick into the nearest slot with the left arm.
  2: frames 711-1114: Press the memory module with both arms in coordination.


In [26]:
# # Extract Video Frames

# frames_dir, total_frames, fps = extract_all_frames(video_path_head)
# frames_dir, total_frames, fps = extract_all_frames(video_path_hand_right)
# frames_dir, total_frames, fps = extract_all_frames(video_path_hand_left)

In [34]:
# Get action config from episode_info (already loaded in your notebook)
action_config = episode_info.get('label_info', {}).get('action_config', [])
task_name = episode_info['task_name']

# Choose which action transition you want (e.g., action 1)
action_idx = 1
target_frame = 500#action_config[action_idx]['start_frame']

In [36]:
# Combine task name with previous and current action prompts
if action_idx > 0:
    prev_action_text = action_config[action_idx - 1]['action_text']
    curr_action_text = action_config[action_idx]['action_text']
    combined_prompt = f"{task_name}: {prev_action_text}, {curr_action_text}"
else:
    # If it's the first action, just use task name + current action
    curr_action_text = action_config[action_idx]['action_text']
    combined_prompt = f"{task_name}: {curr_action_text}"

print(f"Combined prompt: {combined_prompt}")

# Create sample folder
create_sample_folder(
    episode_dir=f"/shared_work/physical_intelligence/agibot_world_sample/{TASK_ID}/{episode_id}",
    output_dir=f"/shared_work/physical_intelligence/ruiheng/Genie-Envisioner/video_gen_examples/sample_{episode_id}_frame{target_frame}",
    target_frame=target_frame,
    n_frames=4,
    frame_spacing=25,
    prompt=combined_prompt
)

Combined prompt: Install memory module: Press the latch of the memory card slot with your right arm., Insert the memory stick into the nearest slot with the left arm.
Target frame: 500
Selected memory frames: [425, 450, 475, 500]
Copied 000425.png -> /shared_work/physical_intelligence/ruiheng/Genie-Envisioner/video_gen_examples/sample_684445_frame500/head_color/0.png
Copied 000450.png -> /shared_work/physical_intelligence/ruiheng/Genie-Envisioner/video_gen_examples/sample_684445_frame500/head_color/1.png
Copied 000475.png -> /shared_work/physical_intelligence/ruiheng/Genie-Envisioner/video_gen_examples/sample_684445_frame500/head_color/2.png
Copied 000500.png -> /shared_work/physical_intelligence/ruiheng/Genie-Envisioner/video_gen_examples/sample_684445_frame500/head_color/3.png
Copied 000425.png -> /shared_work/physical_intelligence/ruiheng/Genie-Envisioner/video_gen_examples/sample_684445_frame500/hand_left_color/0.png
Copied 000450.png -> /shared_work/physical_intelligence/ruiheng/G

In [35]:
episode_dir = f"/shared_work/physical_intelligence/agibot_world_sample/{TASK_ID}/{episode_id}"
output_video = f"/shared_work/physical_intelligence/agibot_world_sample/{TASK_ID}/{episode_id}/multiview_frame{target_frame}.mp4"

create_multiview_video(
    episode_dir=episode_dir,
    output_path=output_video,
    start_frame=target_frame,
    n_frames=4,
    frame_spacing=25,
    future_frames=100,
    fps=30
)

Creating video from frames 400 to 600
Total frames: 201
Output resolution: 1920x480

Video saved to: /shared_work/physical_intelligence/agibot_world_sample/429/684445/multiview_frame500.mp4
